In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.ticker as ticker
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
base_path = '../logs_parsed'
plot_path = '../plots'

In [ ]:
def plot_single_stat(dfs, cfgs, plot_x, plot_y, alpha=0.9, use_rolling_window=False, window_size=10, 
                     show_legend=True, show_title=True, title=None, show_grid=True, filename=None, 
                     x_label=None, y_label=None):
    fig = mpl.figure(figsize=(12, 3))
    for cfg in cfgs:
        y_data = dfs[cfg][plot_y]
        if use_rolling_window:
            y_data = y_data.rolling(window_size).mean()
        mpl.plot(dfs[cfg][plot_x], y_data, label=cfg, alpha=alpha)
    if show_legend:
        mpl.legend(ncols=10)
    if show_grid:
        mpl.grid(visible=True, which='both', axis='both')
    if show_title:
        title = plot_y if not title else title
        mpl.title(title, fontdict=None, loc='center', pad=None)
    if x_label:
        mpl.xlabel(x_label)
    if y_label:
        mpl.ylabel(y_label)
    fig.axes[0].xaxis.set_major_formatter(ticker.EngFormatter())
    mpl.show()    
    if filename:
        fig.savefig(filename)
        
def plot_multiple_vertically(data, cfgs, rows, plot_x, alpha=0.9, use_rolling_window=False, window_size=10, 
                         show_legend=True, title=None, show_grid=True, filename=None, x_label=None):
    fig, axis = mpl.subplots(len(rows), figsize=(12, 3*len(rows)), layout="constrained", sharex=True)
    
    for idx, row in enumerate(rows):
        ax = axis[idx]
        ax.set_title(row, loc='left')
        for cfg in cfgs:
            y_data = data[cfg][row]
            x_data = data[cfg][plot_x]
            if use_rolling_window:
                y_data = y_data.rolling(window_size).mean()
            ax.plot(x_data, y_data, label=cfg, alpha=alpha)
        if show_grid:
            ax.grid(visible=True, which='both', axis='both')
        
    if show_legend:
        ax.legend(ncols=10) # calling only once for last subplot
    if title:
        fig.suptitle(title)
    if x_label:
        mpl.xlabel(x_label)
        # fig.set_title(title, fontdict=None, loc='center', pad=None)
    ax.xaxis.set_major_formatter(ticker.EngFormatter())
    mpl.show()
    if filename:
        fig.savefig(filename)

# A2C

In [ ]:
model_name = 'A2C'
configurations = ['V213', 'V219', 'V221']
# configurations = ['V219', 'V220', 'V221']
# configurations = [f"V2{x:02.0f}" for x in range(1,22)]

plot_cfgs = configurations

data = {}
for config in configurations:
    data[config] = pd.read_csv(f"{base_path}/{model_name}_{config}.csv", index_col=0)

In [ ]:
data[configurations[0]].columns

In [ ]:
plot_single_stat(data, plot_cfgs, 'episode', 'eval_sharpe', use_rolling_window=True, window_size=25, alpha=0.8,
                 title=f"{model_name}: Sharpe Ratio on Validation Dataset", x_label='trained episodes', y_label='sharpe ratio', 
                 filename=f"{plot_path}/1D/{model_name}_sharpe.png")

In [ ]:
'''
Columns: ['step', 'episode', 'episode-reward', 'missed_trades', 'sharpe',
       'sortino', 'trades', 'fps', 'entropy_loss', 'explained_variance',
       'learning_rate', 'policy_loss', 'reward', 'std', 'value_loss',
       'wall_time', 'eval_episode_rewards', 'eval_missed_trades',
       'eval_sharpe', 'eval_sortino', 'eval_trades', 'date']
'''

In [ ]:
info_rows = ['eval_sharpe', 'eval_episode_rewards', 'eval_trades']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=25,
                         x_label="trained episodes", title=f"{model_name}: Performance of Test Dataset",
                        filename=f"{plot_path}/1D/{model_name}_performance_test.png")

In [ ]:
info_rows = ['entropy_loss', 'explained_variance', 'policy_loss', 'reward', 'std', 'value_loss']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=25,
                         x_label="trained episodes", title=f"{model_name}: Training Metrics",
                        filename=f"{plot_path}/1D/{model_name}_training_metrics.png")

In [ ]:
info_rows = ['sharpe', 'episode-reward', 'trades']  # , 'eval_missed_trades'
plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=25,
                         x_label="trained episodes", title=f"{model_name}: Performance of Training Dataset",
                         filename=f"{plot_path}/1D/{model_name}_performance_train.png")

# PPO

In [ ]:
model_name = 'PPO'
# configurations = ['V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208']
configurations = ['V205',  'V207', 'V208']
plot_cfgs = configurations

data = {}
for config in configurations:
    data[config] = pd.read_csv(f"{base_path}/{model_name}_{config}.csv", index_col=0)

In [ ]:
data[configurations[0]].columns

In [ ]:
plot_single_stat(data, plot_cfgs, 'episode', 'eval_sharpe', use_rolling_window=True, window_size=5, alpha=0.8,
                 title=f"{model_name}: Sharpe Ratio on Validation Dataset", x_label='trained episodes', y_label='sharpe ratio', 
                 filename=f"{plot_path}/1D/{model_name}_sharpe.png")

In [ ]:
info_rows = ['eval_sharpe', 'eval_episode_rewards', 'eval_trades']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Test Dataset",
                        filename=f"{plot_path}/1D/{model_name}_performance_test.png")

In [ ]:
info_rows = ['approx_kl', 'clip_fraction', 'clip_range', 'entropy_loss', 'explained_variance', 'loss', 'policy_gradient_loss', 'reward', 'std', 'value_loss']

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=10,
                         x_label="trained episodes", title=f"{model_name}: Training Metrics",
                        filename=f"{plot_path}/1D/{model_name}_training_metrics.png")

In [ ]:
info_rows = ['sharpe', 'episode-reward', 'trades']  # , 'eval_missed_trades'
plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Training Dataset",
                         filename=f"{plot_path}/1D/{model_name}_performance_train.png")

# TD3

In [ ]:
model_name = 'TD3'
# configurations = ['V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217']
configurations = ['V203', 'V206', 'V213', 'V214']
plot_cfgs = configurations

data = {}
for config in configurations:
    data[config] = pd.read_csv(f"{base_path}/{model_name}_{config}.csv", index_col=0)

In [ ]:
data[configurations[0]].columns

In [ ]:
plot_single_stat(data, plot_cfgs, 'episode', 'eval_sharpe', use_rolling_window=True, window_size=5, alpha=0.8,
                 title=f"{model_name}: Sharpe Ratio on Validation Dataset", x_label='trained episodes', y_label='sharpe ratio', 
                 filename=f"{plot_path}/1D/{model_name}_sharpe.png")

In [ ]:
info_rows = ['eval_sharpe', 'eval_episode_rewards', 'eval_trades']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Test Dataset",
                        filename=f"{plot_path}/1D/{model_name}_performance_test.png")

In [ ]:
info_rows = ['actor_loss', 'critic_loss', 'reward']

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Training Metrics",
                        filename=f"{plot_path}/1D/{model_name}_training_metrics.png")

In [ ]:
info_rows = ['sharpe', 'episode-reward', 'trades']  # , 'eval_missed_trades'
plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Training Dataset",
                         filename=f"{plot_path}/1D/{model_name}_performance_train.png")

# TD3 Best Model - Different Timeframes

In [ ]:
model_name = 'TD3'
# configurations = ['V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217']
configurations = ['V214_1D','V214_12H','V214_6H','V214_1H']
plot_cfgs = configurations

data = {}
for config in configurations:
    data[config] = pd.read_csv(f"{base_path}/{model_name}_{config}.csv", index_col=0)

In [ ]:
plot_single_stat(data, plot_cfgs, 'episode', 'eval_sharpe', use_rolling_window=True, window_size=5, alpha=0.8,
                 title=f"{model_name}: Sharpe Ratio on Validation Dataset", x_label='trained episodes', y_label='sharpe ratio', 
                 filename=f"{plot_path}/TD3/{model_name}_sharpe.png")

In [ ]:
info_rows = ['eval_sharpe', 'eval_episode_rewards', 'eval_trades']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Test Dataset",
                        filename=f"{plot_path}/TD3/{model_name}_performance_test.png")

In [ ]:
info_rows = ['actor_loss', 'critic_loss', 'reward']

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Training Metrics",
                        filename=f"{plot_path}/TD3/{model_name}_training_metrics.png")

In [ ]:
info_rows = ['sharpe', 'episode-reward', 'trades']  # , 'eval_missed_trades'
plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Training Dataset",
                         filename=f"{plot_path}/TD3/{model_name}_performance_train.png")

# A2C Best Model - Different Timeframes

In [ ]:
model_name = 'A2C'
# configurations = ['V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217']
configurations = ['V221_1D','V221_12H','V221_6H','V221_1H']
plot_cfgs = configurations

data = {}
for config in configurations:
    data[config] = pd.read_csv(f"{base_path}/{model_name}_{config}.csv", index_col=0)

In [ ]:
plot_single_stat(data, plot_cfgs, 'episode', 'eval_sharpe', use_rolling_window=True, window_size=5, alpha=0.8,
                 title=f"{model_name}: Sharpe Ratio on Validation Dataset", x_label='trained episodes', y_label='sharpe ratio', 
                 filename=f"{plot_path}/A2C/{model_name}_sharpe.png")

In [ ]:
info_rows = ['eval_sharpe', 'eval_episode_rewards', 'eval_trades']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Test Dataset",
                        filename=f"{plot_path}/A2C/{model_name}_performance_test.png")

In [ ]:
info_rows = ['entropy_loss', 'explained_variance', 'policy_loss', 'reward', 'std', 'value_loss']

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Training Metrics",
                        filename=f"{plot_path}/A2C/{model_name}_training_metrics.png")

In [ ]:
info_rows = ['sharpe', 'episode-reward', 'trades']  # , 'eval_missed_trades'
plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Training Dataset",
                         filename=f"{plot_path}/A2C/{model_name}_performance_train.png")

# PPO Best Model - Different Timeframes

In [ ]:
model_name = 'PPO'
# configurations = ['V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217']
configurations = ['V208_1D','V208_12H','V208_6H','V208_1H']
plot_cfgs = configurations

data = {}
for config in configurations:
    data[config] = pd.read_csv(f"{base_path}/{model_name}_{config}.csv", index_col=0)

In [ ]:
plot_single_stat(data, plot_cfgs, 'episode', 'eval_sharpe', use_rolling_window=True, window_size=5, alpha=0.8,
                 title=f"{model_name}: Sharpe Ratio on Validation Dataset", x_label='trained episodes', y_label='sharpe ratio', 
                 filename=f"{plot_path}/PPO/{model_name}_sharpe.png")

In [ ]:
info_rows = ['eval_sharpe', 'eval_episode_rewards', 'eval_trades']  # , 'eval_missed_trades'

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Test Dataset",
                        filename=f"{plot_path}/PPO/{model_name}_performance_test.png")

In [ ]:
info_rows = ['approx_kl', 'clip_fraction', 'clip_range', 'entropy_loss', 'explained_variance', 'loss', 'policy_gradient_loss', 'reward', 'std', 'value_loss']

plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Training Metrics",
                        filename=f"{plot_path}/PPO/{model_name}_training_metrics.png")

In [ ]:
info_rows = ['sharpe', 'episode-reward', 'trades']  # , 'eval_missed_trades'
plot_multiple_vertically(data, plot_cfgs, info_rows, 'episode', use_rolling_window=True, window_size=5,
                         x_label="trained episodes", title=f"{model_name}: Performance of Training Dataset",
                         filename=f"{plot_path}/PPO/{model_name}_performance_train.png")